In [1]:
from datetime import datetime
import numpy as np
import pandas as pd
import tensorflow as tf

import sys
sys.path.append('../')

from wavy import panel
from wavy import side
from wavy import block
from wavy import model
from wavy import plot

In [2]:
dir(pd.DataFrame)

['T',
 '_AXIS_LEN',
 '_AXIS_NAMES',
 '_AXIS_NUMBERS',
 '_AXIS_ORDERS',
 '_AXIS_REVERSED',
 '_AXIS_TO_AXIS_NUMBER',
 '_HANDLED_TYPES',
 '__abs__',
 '__add__',
 '__and__',
 '__annotations__',
 '__array__',
 '__array_priority__',
 '__array_ufunc__',
 '__array_wrap__',
 '__bool__',
 '__class__',
 '__contains__',
 '__copy__',
 '__deepcopy__',
 '__delattr__',
 '__delitem__',
 '__dict__',
 '__dir__',
 '__divmod__',
 '__doc__',
 '__eq__',
 '__finalize__',
 '__floordiv__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__getitem__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__iadd__',
 '__iand__',
 '__ifloordiv__',
 '__imod__',
 '__imul__',
 '__init__',
 '__init_subclass__',
 '__invert__',
 '__ior__',
 '__ipow__',
 '__isub__',
 '__iter__',
 '__itruediv__',
 '__ixor__',
 '__le__',
 '__len__',
 '__lt__',
 '__matmul__',
 '__mod__',
 '__module__',
 '__mul__',
 '__ne__',
 '__neg__',
 '__new__',
 '__nonzero__',
 '__or__',
 '__pos__',
 '__pow__',
 '__radd__',
 '__rand__',
 '__rdivmo

In [3]:
df = pd.read_pickle('../data/processed.pkl')

In [4]:
df.head(5)

LNC                                                    MAS  \
                 Open       High        Low      Close     Volume       Open   
Date                                                                           
2005-12-21  38.127885  39.324626  38.120589  39.207870  2190100.0  18.991695   
2005-12-22  39.185984  39.324630  38.879500  39.040039  1095500.0  19.085703   
2005-12-23  39.061931  39.397604  38.959771  39.324631   588800.0  18.954090   
2005-12-27  39.514374  39.514374  38.981676  39.237080   932700.0  18.929008   
2005-12-28  39.455973  39.696782  39.244353  39.310028   427200.0  18.853798   

                                                        ...       CSX  \
                 High        Low      Close     Volume  ...      Open   
Date                                                    ...             
2005-12-21  19.154659  18.903944  19.085712  1384200.0  ...  6.245725   
2005-12-22  19.298810  18.910202  18.929007  1576900.0  ...  6.381336   
2005-12-23  18.954090  18.684571  18.816196  2090500.0  ...  6.297691   
2005-12-27  18.991687  18.803650  18.847525  1609000.0  ...  6.520750   
2005-12-28  18.935281  18.747245  18.803656  2026300.0  ...  6.419358   

                                                            CI             \
                High       Low     Close     Volume       Open       High   
Date                                                                        
2005-12-21  6.399080  6.245725  6.362326  8358000.0  36.945102  37.462575   
2005-12-22  6.420626  6.361058  6.410487  5344800.0  37.409841  37.630677   
2005-12-23  6.520752  6.297691  6.468790  3630000.0  37.376876  37.432909   
2005-12-27  6.527087  6.369931  6.400348  4905000.0  37.195584  37.455973   
2005-12-28  6.459914  6.363592  6.419358  3578400.0  37.241735  37.449383   

                                             
                  Low      Close     Volume  
Date                                         
2005-12-21  36.895658  37.370289  2805900.0  
2005-12-22  37.073647  37.327442  2296800.0  
2005-12-23  36.905546  37.261517  1686000.0  
2005-12-27  36.961570  37.116482  1917900.0  
2005-12-28  37.109892  37.251621  1248900.0  

[5 rows x 2085 columns]

In [5]:
block.Block.__dict__

mappingproxy({'__module__': 'wavy.block',
              '__init__': <function wavy.block.Block.__init__(self, df, *args, **kwargs)>,
              '_check_method': <function wavy.block.Block._check_method(self, method)>,
              '_constructor': <property at 0x130cf5c20>,
              '_constructor_sliced': <property at 0x130cfe220>,
              'start': <property at 0x130cfe270>,
              'end': <property at 0x130cfe2c0>,
              'assets': <property at 0x130cfe310>,
              'channels': <property at 0x130cfe360>,
              'timesteps': <property at 0x130cfe3b0>,
              'tensor': <property at 0x130cfe400>,
              'matrix': <property at 0x130cfe450>,
              'filter': <function wavy.block.Block.filter(self, assets: List[str] = None, channels: List[str] = None)>,
              '_filter_assets': <function wavy.block.Block._filter_assets(self, assets)>,
              '_filter_channels': <function wavy.block.Block._filter_channels(self, channe

In [6]:
panel_ = panel.from_data(df, lookback = 2, horizon = 1, gap = 0, assets = ['AAPL', 'MSFT'], channels= ['Open', 'Close'])

In [7]:
block = panel_.x[0]

In [8]:
block

AAPL                 MSFT           
                Open     Close       Open      Close
Date                                                
2005-12-21  2.218566  2.246069  19.577126  19.475122
2005-12-22  2.258598  2.261960  19.460543  19.373114

In [9]:
block.wcount()

AAPL       MSFT      
           Open Close Open Close
2005-12-21    2     2    2     2

In [10]:
block.wcount(axis=1)

,asset
,count
Date,
2005-12-21,4
2005-12-22,4


In [11]:
block.wkurt(axis=0)

AAPL       MSFT      
           Open Close Open Close
2005-12-21  NaN   NaN  NaN   NaN

In [12]:
block.wkurt(axis=1)

,asset
,kurt
Date,
2005-12-21,-5.99944
2005-12-22,-5.99961


In [13]:
block.wkurtosis(axis=0)

AAPL       MSFT      
           Open Close Open Close
2005-12-21  NaN   NaN  NaN   NaN

In [14]:
block.wkurtosis(axis=1)

,asset
,kurtosis
Date,
2005-12-21,-5.99944
2005-12-22,-5.99961


In [15]:
block.wmad(axis=0)

AAPL                MSFT          
                Open     Close      Open     Close
2005-12-21  0.020016  0.007946  0.058291  0.051004

In [16]:
block.wmad(axis=1)

,asset
,mad
Date,
2005-12-21,8.646903
2005-12-22,8.578275


In [17]:
block.wmax(axis=0)

AAPL                MSFT           
                Open    Close       Open      Close
2005-12-21  2.258598  2.26196  19.577126  19.475122

In [18]:
block.wmax(axis=1)

,asset
,max
Date,
2005-12-21,19.577126
2005-12-22,19.460543


In [19]:
block.wmin(axis=0)

AAPL                 MSFT           
                Open     Close       Open      Close
2005-12-21  2.218566  2.246069  19.460543  19.373114

In [20]:
block.wmean()

AAPL                 MSFT           
                Open     Close       Open      Close
2005-12-21  2.238582  2.254014  19.518834  19.424118

In [21]:
block.wmedian()

AAPL                 MSFT           
                Open     Close       Open      Close
2005-12-21  2.238582  2.254014  19.518834  19.424118

In [22]:
block.wnunique(axis=0)

AAPL       MSFT      
           Open Close Open Close
2005-12-21    2     2    2     2

In [23]:
block.wprod()

AAPL                  MSFT           
                Open     Close        Open      Close
2005-12-21  5.010849  5.080517  380.981498  377.29376

In [24]:
block.wquantile()

AAPL                 MSFT           
                Open     Close       Open      Close
2005-12-21  2.238582  2.254014  19.518834  19.424118

In [25]:
block.wsem()

AAPL                MSFT          
                Open     Close      Open     Close
2005-12-21  0.020016  0.007946  0.058291  0.051004

In [26]:
block.wskew(axis=1)

,asset
,skew
Date,
2005-12-21,0.000084
2005-12-22,0.000067


In [27]:
block.wstd()

AAPL                MSFT          
                Open     Close      Open     Close
2005-12-21  0.028307  0.011237  0.082436  0.072131

In [28]:
block.wsum()

AAPL                 MSFT           
                Open     Close       Open      Close
2005-12-21  4.477164  4.508029  39.037669  38.848236

In [29]:
block.wvar()

AAPL                MSFT          
                Open     Close      Open     Close
2005-12-21  0.000801  0.000126  0.006796  0.005203

In [ ]:
callable(df.fillna)

In [ ]:
hasattr(block.Block, 'start')

In [ ]:
hasattr(block.Block, 'start') and callable(getattr(block.Block, 'start'))

In [ ]:
callable(getattr(block.Block, 'start'))

In [ ]:
hasattr(pd.DataFrame, 'T')

In [ ]:
hasattr(block.Block, 'T')

In [ ]:
hasattr(pd.DataFrame, 'T') and not hasattr(block.Block, 'T')

In [ ]:
dir(block.Block)

In [ ]:
a = panel_.x[0]

In [ ]:
super()

In [ ]:
a.__class__.__dict__

In [ ]:
a.magicmethod('T')

In [ ]:
a.add(1)

In [ ]:
a.mean()

In [ ]:
a.min()

In [ ]:
a.max()

In [ ]:
a.mad()

In [ ]:
dir(pd.DataFrame)

In [ ]:
type(df.memory_usage())

In [ ]:
type(a.mean())

In [ ]:
for c in block.super():
    print(c)

In [ ]:
panel_.plot_block(12)

In [ ]:
panel_.plot_slider(20)

In [ ]:
panel_.x.plot_slider(20)

In [ ]:
panel_.x.plot_block(20)

In [ ]:
panel_.x[12].plot()

In [ ]:
new = panel_.x.side_sample(frac=0.01)

In [ ]:
new[0]

In [ ]:
len(new)

In [ ]:
new_panel = panel_.panel_sample(100)

In [ ]:
len(new_panel)

In [ ]:
panel_.x[0].min()

In [ ]:
panel_.x[0].mean().to_frame()

In [ ]:
type(panel_.x[0].mean().to_frame())

In [ ]:
a = pd.DataFrame([1,2,3,4,5])

In [ ]:
type(a.mean())

In [ ]:
# Implement to_frame().T in Block functions that return series (Gabriel)
# Resample 
# Event tag for plots

In [ ]:
block_ = panel_.x[0]

In [ ]:
block_.mean()

In [ ]:
panel_.x[0].mean()

In [ ]:
panel_.x.min()

In [ ]:
type(panel_.x[0].mean().to_frame().T)

In [ ]:
panel_.x

In [ ]:
panel_.x.first

In [ ]:
panel_.plot_slider(100)

In [ ]:
panel_.plot_block(20)

In [ ]:
panel_.x.plot_slider()

# Test functions example

In [ ]:
side = panel_.x

In [ ]:
side[0]

In [ ]:
side = side.side_shift(window = 1)

In [ ]:
side[0]

In [ ]:
side[0].is_valid

In [ ]:
side[1]

In [ ]:
side = side.side_diff(window = 1)

In [ ]:
side[1]

In [ ]:
side = side.side_pct_change(window = 1)

In [ ]:
side[1]

# Graph types

Separate X from Y

In [ ]:
panel_.plot(2)

In [ ]:
panel_.plot(3)

In [ ]:
panel_.x.first

In [ ]:
panel_.x[0]

In [ ]:
panel.x[0].index

In [ ]:
panel_.x.plot(2)

In [ ]:
panel.x[0]

In [ ]:
panel_.x[2].plot()

In [ ]:
panel.y[0]

In [ ]:
panel.set_training_split()

In [ ]:
panel.x.first

In [ ]:
panel.shape

# Baseline

In [ ]:
baseline = nn.Baseline(panel, metrics=['SE', 'MSE', 'RMSE'])

In [ ]:
predicted = baseline.predict()

In [ ]:
panel.test.y[0]

In [ ]:
predicted[2]

In [ ]:
predicted_val = baseline.predict_val()

In [ ]:
panel.val.y[0]

In [ ]:
predicted_val[2]

In [ ]:
baseline.evaluate('test')

In [ ]:
# m = tf.keras.metrics.MeanSquaredError()
# m.update_state(panel.test.y.tensor3d, predicted.tensor3d)
# m.result().numpy()

# DenseModel

In [ ]:
densemodel = nn.DenseModel(panel, model_type="regression")

In [ ]:
densemodel.fit()

In [ ]:
densemodel.model.summary()

In [ ]:
predicted = densemodel.predict()

In [ ]:
predicted[0]

In [ ]:
panel.test.y[0]

In [ ]:
densemodel.evaluate('test')

In [ ]:
import numpy as np
import pandas as pd
import plotly as px
import plotly.graph_objects as go
import plotly.express as px

pd.set_option("multi_sparse", True)  # To see multilevel indexes
pd.options.plotting.backend = "plotly"

from plotly.subplots import make_subplots

def pair_plot(panel, idx):

    cmap = px.colors.qualitative.Plotly

    fig = make_subplots(rows=len(panel.channels), cols=len(panel.assets), subplot_titles=panel.assets, shared_xaxes=True)

    for j, channel in enumerate(panel.channels):
        c = cmap[j]
        for i, asset in enumerate(panel.assets):

            showlegend = i <= 0
            x_df = panel.x[idx].filter(assets=asset, channels=channel)
            y_df = panel.y[idx].filter(assets=asset, channels=channel)
            # y_df = panel.y[idx].filter(assets=asset, channels=channel)

            x_trace = go.Scatter(x=x_df.index, y=x_df.values.flatten(),
                            line=dict(width=2, color=c), showlegend=showlegend, name=channel)

            y_trace = go.Scatter(x=y_df.index, y=y_df.values.flatten(),
                                line=dict(width=2, dash='dot', color=c), showlegend=False)

            fig.add_trace(x_trace, row=j+1, col=i+1)
            fig.add_trace(y_trace, row=j+1, col=i+1)

    fig.update_layout(showlegend=True)
    fig.show()

In [ ]:
panel.x.as_dataframe()

In [ ]:
# panel = panel.filter('AAPL', ['High', 'Low'])
pair_plot(panel, 0)

In [ ]:
panel = panel.filter('LNC', ['Open', 'High'])
panel[0].plot()
panel.y_ = model.predict()

# ConvModel

In [ ]:
convmodel = nn.ConvModel(panel, model_type="regression")

In [ ]:
convmodel.fit()

In [ ]:
convmodel.model.summary()

In [ ]:
predicted = convmodel.predict()

In [ ]:
predicted[0]

In [ ]:
panel.test.y[0]

In [ ]:
convmodel.evaluate()

# LinearRegression

In [ ]:
linearRegression = nn.LinearRegression(panel)

In [ ]:
linearRegression.fit()

In [ ]:
predicted = linearRegression.predict()

In [ ]:
predicted[0]

In [ ]:
panel.test.y[0]

In [ ]:
linearRegression.evaluate()

# Teste SeparateAssetModel

In [ ]:
panel.val.x[0]

In [ ]:
test_splits = panel.val.x._split_assets()

In [ ]:
x_train = [x.tensor3d for x in test_splits]

In [ ]:
# pd.DataFrame(x_train)

In [ ]:
test_splits[0].tensor3d[0]

In [ ]:
test_splits[0].tensor4d[0]

In [ ]:
pd.DataFrame(panel.val.x.tensor4d)

# SeparateAssetModel

In [ ]:
separateAssetModel = nn.SeparateAssetModel(panel, loss='MSE', optimizer='adam', metrics=['MAE'], hidden_activation='relu', last_activation='linear')

In [ ]:
separateAssetModel.fit()

In [ ]:
separateAssetModel.model.summary()

In [ ]:
predicted = separateAssetModel.predict()

In [ ]:
predicted[0]

In [ ]:
panel.test.y[0]

In [ ]:
separateAssetModel.evaluate()

# SeparateAssetConvModel

In [ ]:
separateAssetConvModel = nn.SeparateAssetConvModel(panel, loss='MSE', optimizer='adam', metrics=['MAE'], hidden_activation='relu', last_activation='linear')

In [ ]:
separateAssetConvModel.fit()

In [ ]:
separateAssetConvModel.model.summary()

In [ ]:
predicted = separateAssetConvModel.predict()

In [ ]:
predicted[0]

In [ ]:
panel.test.y[0]

In [ ]:
separateAssetConvModel.evaluate()

# Test Docstrings

In [ ]:
densemodel = nn.DenseModel()